# Control allocation example for Cybership Jonny 

For the sake of testing, set `PYTHONPATH` so notebook can be run without installing the package.

In [1]:
import sys
sys.path.insert(0, "../src")

First, import the control allocation packge

In [2]:
import skadipy
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
# Use interactive backend
%matplotlib qt5

Now, we describe the thrusters. There are different options for thrusters. These are `Fixed`, `Aximuth`, `Vectored`.
It is defined by the position of the thruster and the direction that its directed at.
Orientation of the thruster is defined using quaternions.

Depending on the allocator choice, `extra_attributes` dictionary changes.
For Cybership Jonny, we use MinimumMagnitude and Azimuth Rate allocator. It takes `saturation_limit` and `rate_limit` values to set thrust limits for the thrusters.

In [3]:
tunnel = skadipy.actuator.Fixed(
    position=skadipy.toolbox.Point([0.3875, 0.0, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), radians=np.pi / 2.0),
    extra_attributes={
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "name": "tunnel",
    }
)
port_azimuth = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([-0.4574, -0.055, 0.0]),
    extra_attributes={
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "reference_angle":  np.pi / 4.0,
        "name": "port_azimuth",
    }
)
starboard_azimuth = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([-0.4547, 0.055, 0.0]),
    extra_attributes={
        "rate_limit": 1.0,
        "saturation_limit": 1.0,
        "reference_angle": - np.pi / 4.0,
        "name": "starboard_azimuth",
    }
)

# Put all actuators in a list and create the allocator object
actuators = [
    tunnel,
    port_azimuth,
    starboard_azimuth,
]

Now we need to describe what degrees of freedom we want to control.
To specify that, we use `ForceTorqueComponent` class.
Possible options are

- `ForceTorqueCompontent.X` for surge
- `ForceTorqueCompontent.Y` for sway
- `ForceTorqueCompontent.Z` for heave
- `ForceTorqueCompontent.K` for roll
- `ForceTorqueCompontent.M` for pitch
- `ForceTorqueCompontent.N` for yaw


In [4]:
# List all the degrees of freedom that we want to control
dofs = [
    skadipy.allocator._base.ForceTorqueComponent.X,
    skadipy.allocator._base.ForceTorqueComponent.Y,
    skadipy.allocator._base.ForceTorqueComponent.N
]

Next, we create the allocator object using the thrusters and the degrees of freedom we want to control.

In [5]:
import skadipy.allocator.reference_filters

# Create the allocator object
allocator = skadipy.allocator.reference_filters.MinimumMagnitudeAndAzimuth(
    actuators=actuators,
    force_torque_components=dofs,
    gamma=0.1,
    mu=0.1,
    rho=100,
    time_step=0.1,
    control_barrier_function=skadipy.safety.ControlBarrierFunctionType.ABSOLUTE
)

To allow change in vehicle configuration, allocation matrix can be updated using `update_allocation_matrix` method.

Finally, we can use the allocator to allocate the forces and torques to the thrusters.

In [6]:
# Compute or update the configuration matrix
allocator.compute_configuration_matrix()

Regardless of the DOF we want to control, the allocator will always return forces and torques in the body frame.
If we want to control the vehicle in the NED frame, we need to rotate the forces and torques to the NED frame.

The $\tau_{\text{cmd}}$ **must** be in the form of

$$
\tau_{\text{cmd}} = \begin{bmatrix} F_x & F_y & F_z & M_x & M_y & M_z \end{bmatrix}
$$

where $F_x, F_y, F_z$ are the forces in the body frame and $M_x, M_y, M_z$ are the torques in the body frame.

In [7]:
tau_cmd = np.array(
    [[1.0],
     [0],
     [0],
     [0],
     [0],
     [0.0]], dtype=np.float32)

# Allocate a control signal
allocator.allocate(tau=tau_cmd)

# Get the allocated force
print(f"Allocated {allocator.allocated}")

Allocated [[1.16666667e+00]
 [8.67361738e-19]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]


To get commanded forces to the thrusters, we can use the actuator objects.

In [8]:
for actuator in actuators:
    print(f"{actuator.extra_attributes['name']}: {actuator.force}")

tunnel: [[2.01128555e-05]]
port_azimuth: [[0.5833353 ]
 [0.00635386]]
starboard_azimuth: [[ 0.58333137]
 [-0.00637397]]


In [9]:
# Create sliders for F_x, F_y, and M_z
F_x_slider = widgets.FloatSlider(value=1.0, min=-10.0, max=10.0, step=0.1, description='F_x:')
F_y_slider = widgets.FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1, description='F_y:')
M_z_slider = widgets.FloatSlider(value=1.0, min=-10.0, max=10.0, step=0.1, description='M_z:')

# Create a button to trigger the allocator
allocate_button = widgets.Button(description='Allocate')

plt.ion()
fig, ax = plt.subplots(figsize=(10, 10))

# Function to update tau_cmd and allocate forces
def allocate_forces(button):
    global tau_cmd
    tau_cmd[0, 0] = F_x_slider.value
    tau_cmd[1, 0] = F_y_slider.value
    tau_cmd[5, 0] = M_z_slider.value
    allocator.allocate(tau=tau_cmd)

    positions = [actuator.position for actuator in actuators]
    forces = [actuator.force for actuator in actuators]
    # Correct size for tunnel thruster to be size of two

    # Find tunnel thrusters and correct the size of the forces
    for i, actuator in enumerate(actuators):
        if forces[i].shape[0] == 1:
            forces[i] = np.concatenate((forces[i], np.zeros((1,1))), axis=0)
            break

    # Extract X and Y components of positions and forces
    x_positions = [pos.x for pos in positions]
    y_positions = [pos.y for pos in positions]
    x_forces = [force[0] for force in forces]
    y_forces = [force[1] for force in forces]
    # Normalize the forces for better visualization
    scale_factor = 0.2
    x_forces = [force * scale_factor for force in x_forces]
    y_forces = [force * scale_factor for force in y_forces]

    # Create the plot with predefined limits
    ax.cla()
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)
    ax.grid()
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title('Thrust Directions and Magnitudes in X-Y Plane')

    ax.quiver(x_positions, y_positions, x_forces, y_forces, angles='xy', scale_units='xy', scale=1, color='r')
    ax.scatter(x_positions, y_positions, color='b')
    fig.canvas.draw_idle()


# Attach the function to the button
allocate_button.on_click(allocate_forces)

# Display the widgets
display(F_x_slider, F_y_slider, M_z_slider, allocate_button)

FloatSlider(value=1.0, description='F_x:', max=10.0, min=-10.0)

FloatSlider(value=0.0, description='F_y:', max=10.0, min=-10.0)

FloatSlider(value=1.0, description='M_z:', max=10.0, min=-10.0)

Button(description='Allocate', style=ButtonStyle())